In [1]:

import xarray as xr
import pandas as pd
import glob
import os
import geopandas as gpd
from shapely.geometry import Point

# Gộp dữ liệu nc4 vào file csv

In [2]:

# Đường dẫn đến thư mục chứa các file .nc4
folder_path = "E:\\Downloads\\oco2\\oco-xco2\\data\\DBSH_2023\\*.nc4"

# Lấy danh sách tất cả các file .nc4
file_list = glob.glob(folder_path)

# Danh sách lưu DataFrame từ mỗi file
df_list = []

for file in file_list:
    # Trích xuất ngày từ tên file
    filename = os.path.basename(file)

    # Tách phần ngày từ tên file (vị trí thứ 2 khi split bằng _)
    date_str = filename.split('_')[2]

    # Chuyển đổi định dạng ngày (240921 -> 2024-09-21)
    full_date = f"20{date_str[:2]}-{date_str[2:4]}-{date_str[4:6]}"

    ds = xr.open_dataset(file)

    # Chuyển đổi dữ liệu thành Pandas DataFrame
    df = pd.DataFrame({
        "file_date": full_date,  # Thêm ngày từ tên file
        "latitude": ds["latitude"].values,
        "longitude": ds["longitude"].values,
        "time": ds["time"].values,
        "xco2": ds["xco2"].values
    })

    df_list.append(df)

# Gộp tất cả dữ liệu lại thành một DataFrame lớn
df_final = pd.concat(df_list, ignore_index=True)

# Lưu ra file CSV chung
df_final.to_csv("CO2_all_2023.csv", index=False)

print("Xuất dữ liệu thành công ra CO2_all_2023.csv!")
print(f"Tổng số file đã xử lý: {len(file_list)}")
print(f"Tổng số dòng dữ liệu: {len(df_final)}")


Xuất dữ liệu thành công ra CO2_all_2023.csv!
Tổng số file đã xử lý: 149
Tổng số dòng dữ liệu: 23081205


# Xuất dữ liệu CO2 từ file CO2_all lọc theo ranh giới VietNam sang file CO2_vietnam

In [3]:

# Đọc file CSV chứa tọa độ các điểm trong khu vực Việt Nam
df = pd.read_csv("CO2_all_2023.csv")

# Đọc shapefile biên giới đất liền Việt Nam
shapefile_path = r"E:\Downloads\oco2\oco-xco2\VNM_adm\VNM_adm0.shp"
gdf_vietnam = gpd.read_file(shapefile_path)

# Chuyển hệ tọa độ shapefile về EPSG:4326 nếu chưa đúng
if gdf_vietnam.crs != "EPSG:4326":
    gdf_vietnam = gdf_vietnam.to_crs(epsg=4326)

# Chuyển DataFrame thành GeoDataFrame
geometry = [Point(xy) for xy in zip(df["longitude"], df["latitude"])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# Lọc sơ bộ theo bounding box của Việt Nam
min_lon, min_lat, max_lon, max_lat = gdf_vietnam.total_bounds
gdf = gdf[(gdf["longitude"] >= min_lon) & (gdf["longitude"] <= max_lon) &
          (gdf["latitude"] >= min_lat) & (gdf["latitude"] <= max_lat)]

# Dùng sjoin để kiểm tra điểm có nằm trong đất liền không
gdf_land = gpd.sjoin(gdf, gdf_vietnam, predicate="within", how="inner")

# Xuất ra CSV (loại bỏ các cột không cần thiết)
gdf_land.drop(columns=["geometry", "index_right"], errors="ignore").to_csv(
    "CO2_vietnam_2023.csv", index=False)

print("Xuất file CO2_vietnam_2023.csv thành công!")


Xuất file CO2_vietnam_2023.csv thành công!


# Xuất dữ liệu CO2 từ file CO2_vietnam lọc theo ranh giới DBSH sang file CO2_DBSH

In [4]:
# Đọc file CSV chứa tọa độ các điểm trong khu vực Việt Nam
df = pd.read_csv("CO2_vietnam_2023.csv")

# Đọc shapefile biên giới đất liền Việt Nam
shapefile_path = r"E:\Downloads\oco2\oco-xco2\ranhGioi_DBSH\dbsh_clip.shp"
gdf_vietnam = gpd.read_file(shapefile_path)

# Chuyển hệ tọa độ shapefile về EPSG:4326 nếu chưa đúng
if gdf_vietnam.crs != "EPSG:4326":
    gdf_vietnam = gdf_vietnam.to_crs(epsg=4326)

# Chuyển DataFrame thành GeoDataFrame
geometry = [Point(xy) for xy in zip(df["longitude"], df["latitude"])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# Lọc sơ bộ theo bounding box của Việt Nam
min_lon, min_lat, max_lon, max_lat = gdf_vietnam.total_bounds
gdf = gdf[(gdf["longitude"] >= min_lon) & (gdf["longitude"] <= max_lon) &
          (gdf["latitude"] >= min_lat) & (gdf["latitude"] <= max_lat)]

# Dùng sjoin để kiểm tra điểm có nằm trong đất liền không
gdf_land = gpd.sjoin(gdf, gdf_vietnam, predicate="within", how="inner")

# Xuất ra CSV (loại bỏ các cột không cần thiết)
gdf_land.drop(columns=["geometry", "index_right"], errors="ignore").to_csv(
    "CO2_DBSH_2023.csv", index=False)

print("Xuất file CO2_DBSH_2023.csv thành công!")


Xuất file CO2_DBSH_2023.csv thành công!
